In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import csv, datetime
import pandas as pd
import matplotlib.dates as mdates
import os, sys
from tqdm import tqdm


def MakeDataFromText(filepath):
    """
    Helper function to parse text files in IFM proprietary format.
    """
    with open(filepath, 'r') as file:
        reader = csv.reader(file)

        db = dict()                          # Dictionary that will hold k,v pairs of 
        recent_EPC = ''                      # The most recent EPC that was read, and data is being assigned to
        Time_Or_Data = True                  # Time is true, Data is false
        reached_24 = False

        for l in reader:                     # Iterate over entire text file
            line = l[0]                      # Each line only has one token, so we select it
            
            if len(line)==24:                # If it is an EPC
                db[line] = [[],[]]           # Create new k,v pair in database
                recent_EPC = line            # Current EPC to add to is selected
            
            elif line == '[':                # If start bracket, beginning to collect an array of data
                data = []

            elif line == ']':                # End bracket means end of data array
                if len(data)!=0:
                    if Time_Or_Data:
                        db[recent_EPC][0] = data
                        Time_Or_Data = False
                    else:
                        db[recent_EPC][1] = data
                        Time_Or_Data = True
                data = []

            elif line.count(":")==2:
                h,m,s = line.split(':')      # Parse timestamp into hours, minutes, seconds

                if int(h)==23:
                    reached_24 = True        # If end of day reached, must move to Day 2

                if reached_24 and int(h) < 12:
                    day=2
                else:
                    day=1

                data.append(datetime.datetime(1970, 2, day, int(h), int(m), int(s)))
            
            elif line ==' ':               # If blank space, then we skip to parse next line
                pass

            else:                          # If it is a data point
                try:
                    data.append(float(line))
                except:
                    pass                   # Error catching case just in case, so annoying errors don't crash program
    return db


r1 = MakeDataFromText("04-14-2023/RSSI_13295.csv")
r2 = MakeDataFromText("04-14-2023/RSSI_56624.csv")
r3 = MakeDataFromText("04-14-2023/RSSI_63331.csv")
t1 = MakeDataFromText("04-14-2023/tags_13295.csv")
t2 = MakeDataFromText("04-14-2023/tags_56624.csv")
t3 = MakeDataFromText("04-14-2023/tags_63331.csv")

temp = dict()
for d in [ t1,t2,t3 ]:
    for k,v in d.items():
        if k in temp: # if tag is already in dict
            for i in range(len(v[0])):
                if v[0][i] in temp[k]:
                    temp[k][v[0][i]].append(v[1][i])
                else:
                    temp[k][v[0][i]] = [v[1][i]]
        else:             # if tag is not in dict
            temp[k] = dict()
            for i in range(len(v[0])):
                temp[k][v[0][i]] = [v[1][i]]
for k,v in temp.items():
    for k2,v2 in v.items():
        temp[k][k2] = sum(v2)/len(v2)

rssi = dict()
for d in [ r1,r2,r3 ]:
    for k,v in d.items():
        if k in rssi: # if tag is already in dict
            for i in range(len(v[0])):
                if v[0][i] in rssi[k]:
                    rssi[k][v[0][i]].append(v[1][i])
                else:
                    rssi[k][v[0][i]] = [v[1][i]]
        else:             # if tag is not in dict
            rssi[k] = dict()
            for i in range(len(v[0])):
                rssi[k][v[0][i]] = [v[1][i]]
for k,v in rssi.items():
    for k2,v2 in v.items():
        rssi[k][k2] = sum(v2)/len(v2)

In [ ]:
therm =  pd.read_csv( 'Smart Thermometer 1_export_202304141517.csv', skiprows=1, sep=',', header=0 )
therm.columns = [ 'Time', 'TempC', 'RelHumid' ]

new = []
for i in range(len(therm['Time'])):
    h,m,s = therm['Time'][i].split(':')
    new.append ( datetime.datetime(1970, 2, 1, int(h[-2:]), int(m), int(s)))

therm['Time'] = new

In [ ]:
previous = []
with open('Previous water temp data.txt', 'r') as file:
    reader = csv.reader(file)
    for l in reader:
        previous.append(float(l[0])) 

In [ ]:
water =  pd.read_csv( '01010019.TXT', skiprows=7, sep=',', header=0 )
water.columns = [ 'Time', 'placeholder', 'TempC' ]

new = []
nt = []
for i in range(len(water['Time'])):
    h,m,s = water['Time'][i].split(':')
    new.append ( datetime.datetime(1970, 2, 1, int(h[-2:]), int(m), int(s)) + datetime.timedelta(hours=10, minutes=40) )
    nt.append(float(water['TempC'][i][:4]))

water['Time'] = new
water['TempC'] = nt
del new, nt

In [ ]:
# fig, (ax1,ax2) = plt.subplots(2, 1, figsize=(15, 14))
# ax1.xaxis.set_major_formatter( mdates.DateFormatter('%H:%M:%S') )
# ax2.xaxis.set_major_formatter( mdates.DateFormatter('%H:%M:%S') )

# ax1.set_title("Temperature")
# ax2.set_title("RSSI")

# color = {
#     '2B42':'#030d9c',
#     '3943':'#0488e1',
#     '3734':'#9a5300',
#     '0B5D':'#ff771c',
# }

# for k,v in temp.items():
#     if k[-4:] in [ "2B42","3943",'3734','0B5D'  ]:
#         st = sorted( temp[k].items() )
#         sr = sorted( rssi[k].items() )
#         dt, tt, dr = [ j[1] for j in st ], [ j[0] for j in st ], [ j[1] for j in sr ]

#         print(k)

#         ax1.plot( tt, dt, label=k[-4:], color=color[k[-4:]], linewidth=0.40 )
#         ax2.plot( tt, dr, label=k[-4:], color=color[k[-4:]], linewidth=0.20 )

# ax1.plot( therm['Time'], therm['TempC'], label='Outside Thermometer', color='black', linewidth=0.90 )
# ax1.plot( water['Time'], water['TempC'], label='Water', color='#028F0A', linewidth=0.90 )
# ax1.legend()
# ax2.legend()

In [ ]:
every = [ datetime.datetime(1970, 2, 1, 11, 0, 0) + datetime.timedelta(seconds=i) for i in range(7*3600) ] 
diff  = [ 0.0 for i in range(7*3600) ]

# Predicted Heat Loss
_2B42data = sorted( temp['E282403E000207D6F9772B42'].items() )
_3943data = sorted( temp['33333333335F5F5F30363943'].items() )
_3734data = sorted( temp['31313131315F5F5F37373734'].items() )
_0B5Ddata = sorted( temp['E282403E000207D6F9770B5D'].items() )

_2B42_tt = [ u[0] for u in _2B42data ]
_2B42_dt = [ u[1] for u in _2B42data ]
_3943_tt = [ u[0] for u in _3943data ]
_3943_dt = [ u[1] for u in _3943data ]
_3734_tt = [ u[0] for u in _3734data ]
_3734_dt = [ u[1] for u in _3734data ]
_0B5D_tt = [ u[0] for u in _0B5Ddata ]
_0B5D_dt = [ u[1] for u in _0B5Ddata ]

new_every, diff = [], []
for t in tqdm(every):
    if t in _3943_tt and t in _3734_tt:
        diff.append(0.78224*(_3943_dt[ _3943_tt.index(t) ] - _3734_dt[ _3734_tt.index(t) ]))
        new_every.append(t)

In [ ]:
previous_time = [ water['Time'][0] + datetime.timedelta(seconds=i) for i in range(len(previous)) ]

ptimeformat   = [ previous_time[i].timestamp() for i in range(len(previous_time)) ]
timeformat    = [ water['Time'][i].timestamp() for i in range(len(water['Time'])) ]

pm    = np.poly1d(np.polyfit(ptimeformat, previous,       2))
model = np.poly1d(np.polyfit(timeformat,  water['TempC'], 2))

previous = [ 4479.0*(pm(ptimeformat)[k+1]   - pm(ptimeformat)[k])  /2.0 for k in range(len(pm(ptimeformat))-1) ]
ahl      = [ 4479.0*(model(timeformat)[i+1] - model(timeformat)[i])/2.0 for i in range(len(model(timeformat))-1) ]

previous = [previous[0]] + previous
ahl = [ahl[0]] + ahl
water['AHL'] = ahl

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 5.5))
ax.set_title("Actual vs Predicted Heat Loss (In=3934, Out=3734)")

ax.plot(water['Time'], -1.0 * water['AHL'], label='Actual', color='#028F0A', linewidth=1.5 )
ax.plot(new_every, diff, label='Predicted', color='#3003FB', linewidth=0.9 )
ax.plot(previous_time, -1.0 * np.array(previous), label='Previous', color='#d10202', linewidth=1.5 )

ax.legend()
ax.xaxis.set_major_formatter( mdates.DateFormatter('%H:%M:%S') )